# PyCHO Geopandas Workshop

In [ ]:
!conda install --yes geopandas descartes folium contextily
#!pip install geopandas descartes folium contextily

In [ ]:
import pandas as pd
import geopandas as gpd
import descartes
import contextily
import matplotlib.pyplot as plt
import folium

# Open Police Data

[OpenPoliceData](https://github.com/openpolicedata/openpolicedata) is a Python package that provides easy access to 282 (and growing) incident-level open datasets from police departments around the United States. Datasets include traffic stops, use of force, officer-involved shootings, complaints, and other types of police interactions.

Users request data by department name and type of data. The data is returned as a pandas DataFrame. There is no need to manually find the data online or make API calls. 

In [ ]:
!pip install openpolicedata

In [ ]:
import openpolicedata as opd

In [ ]:
data = opd.Source(source_name="Virginia")

In [ ]:
data.datasets

In [ ]:
agencies_richmond = data.get_agencies(partial_name="Richmond")
agencies_charlottesville = data.get_agencies(partial_name="Charlottesville")
agencies_virginia_beach = data.get_agencies(partial_name="Virginia Beach")
agencies_alexandria = data.get_agencies(partial_name="Alexandria")
agencies_arlington = data.get_agencies(partial_name="Arlington")
agencies_norfolk = data.get_agencies(partial_name="Norfolk")
agencies_newport_news = data.get_agencies(partial_name="Newport News")
agencies_chesapeake = data.get_agencies(partial_name="Chesapeake")
agencies_roanoke = data.get_agencies(partial_name="Roanoke")
agencies_lynchburg = data.get_agencies(partial_name="Lynchburg")

print(agencies_richmond)
print()
print(agencies_charlottesville)
print()
print(agencies_virginia_beach)
print()
print(agencies_alexandria)
print()
print(agencies_arlington)
print()
print(agencies_norfolk)
print()
print(agencies_newport_news)
print()
print(agencies_chesapeake)
print()
print(agencies_roanoke)
print()
print(agencies_lynchburg)



<b>Just to give you a preview, the table object for each agency's dataset looks like this.</b>

We are getting traffic stops from the dataset. This is the 'table_type='STOPS' part

In [ ]:
agency = "Richmond Police Department"
table = data.load_from_url(year=2021, table_type='STOPS', agency=agency)
print(table)

print()

print(type(table))

In [ ]:
police_data_tables = {}

agencies = ['Richmond Police Department', 'Charlottesville Police Department', 'Virginia Beach Police Department',  'Alexandria Police Department',
           'Norfolk Police Department', 'Newport News Police Department', 'Chesapeake Police Department', 'Roanoke City Police Department', 'Lynchburg Police Department']

for agency in agencies:
    # get police data for each city in the form of a table
    police_data = data.load_from_url(year=2021, table_type='STOPS', agency=agency)
    # add this as key/value pair to dictionary
    police_data_tables[agency] = police_data.table
    
# renaming 'Roanoke City Police Department' to just 'Roanoke Police Department' so that it matches all the others
police_data_tables['Roanoke Police Department'] = police_data_tables.pop('Roanoke City Police Department')
    

<b>To give another example of the table inside this table object, it is a pandas dataframe!</b>

In [ ]:
richmond_df = table.table
print(type(richmond_df))


In [ ]:
richmond_df

In [ ]:
richmond_df['reason_for_stop'].value_counts()

## City of Richmond GIS Data

[From Richmond GeoHub](https://richmond-geo-hub-cor.hub.arcgis.com/)

In [ ]:
# this is the relative path to where the data is stored
my_path = './GIS_Data'  

In [ ]:
richmond_boundary = gpd.read_file(my_path + "/city_boundaries/Richmond_City_Boundary.shp")
richmond_boundary.plot()

### Add a basemap

In [ ]:
# find this code in geopanda docs: https://geopandas.org/en/stable/gallery/plotting_basemap_background.html?highlight=basemap

ax = richmond_boundary.plot(figsize=(10,8), alpha=0.5)
contextily.add_basemap(ax, crs=richmond_boundary.crs)

In [ ]:
roads = gpd.read_file(my_path + '/Richmond_Roads.geojson')
roads.plot()

### Overlay Multiple Layers

You can put things on top of eachother just as you would in matplotlib

In [ ]:
richmond_boundary = richmond_boundary.to_crs(epsg=4326)   # we will talk about this next

fig, ax = plt.subplots(figsize=(10, 8))
richmond_boundary.plot(ax=ax, color="gray", edgecolor='black', alpha=0.6)
roads.plot(ax=ax, color="red")
contextily.add_basemap(ax=ax, crs=richmond_boundary.crs)



# GeoPandas GeoDataFrames

A GeoPandas GeoDataFrame is an extension to the pandas dataframe object. A GeoDataFrame has a 'geometry' column that stores the spatial geometry of the object.

In [ ]:
roads_clip = roads.iloc[:5]
roads_clip

# Coordinate Reference Systems

In [ ]:
richmond_boundary.crs

In [ ]:
richmond_boundary = richmond_boundary.to_crs(epsg=4326)
richmond_boundary.plot()

In [ ]:
schools = gpd.read_file(my_path + '/Richmond_Public_Schools.geojson')

schools['Type'].value_counts()

### Add Legend

Matplotlib is smart enough to put a legend where there is space in the map. You can manually specify where it goes if you want.

In [ ]:

fig, ax = plt.subplots(figsize=(10, 8))
richmond_boundary.plot(ax=ax, color="gray", edgecolor='black', alpha=0.35)
schools.plot(ax=ax, column='Type', categorical=True, legend=True)           # notice the legend!
contextily.add_basemap(ax=ax, crs=schools.crs)

## Bring back the police data

In [ ]:
Richmond = gpd.read_file(my_path + "/city_boundaries/Richmond_City_Boundary.shp")
Richmond = Richmond.to_crs(epsg=4326)

Charlottesville = gpd.read_file(my_path + "/city_boundaries/Charlottesville_City_Boundary.geojson")
Charlottesville = Charlottesville.to_crs(epsg=4326)

Alexandria = gpd.read_file(my_path + "/city_boundaries/Alexandria_City_Boundary.geojson")
Alexandria = Alexandria.to_crs(epsg=4326)

Virginia_Beach = gpd.read_file(my_path + "/city_boundaries/Virginia_Beach_City_Boundary.geojson")
Virginia_Beach = Virginia_Beach.to_crs(epsg=4326)

Norfolk = gpd.read_file(my_path + "/city_boundaries/Norfolk_City_Boundary.geojson")
Norfolk = Norfolk.to_crs(epsg=4326)

Newport_News = gpd.read_file(my_path + "/city_boundaries/Newport_News_City_Boundary.gpkg")
Newport_News = Newport_News.to_crs(epsg=4326)

Chesapeake = gpd.read_file(my_path + "/city_boundaries/Chesapeake_City_Boundary.geojson")
Chesapeake = Chesapeake.to_crs(epsg=4326)

Roanoke = gpd.read_file(my_path + "/city_boundaries/Roanoke_City_Boundary.gpkg")
Roanoke = Roanoke.to_crs(epsg=4326)

Lynchburg = gpd.read_file(my_path + "/city_boundaries/Lynchburg_City_Boundary.geojson")
Lynchburg = Lynchburg.to_crs(epsg=4326)


In [ ]:
cities = [Richmond, Charlottesville, Alexandria, Virginia_Beach, Norfolk, Newport_News, Chesapeake, Roanoke, Lynchburg]

fig, ax = plt.subplots(figsize=(10, 8))
for i in cities:
    i.plot(ax=ax)
contextily.add_basemap(ax=ax, crs=Richmond.crs)

### Clean up police data

In [ ]:
# I had never done this before! Access the global variables currently in memory using the vars() method
# I am doing this because I have a variable above in the 'cities' list with the same name as the police department
# rather than making various lists or dictionaries and then adding them to the GeoDataFrames, I can do it all in one loop!
myVars = vars()     #accesses global variables in memory

# use agency names from police_data_tables dictionary above
for city_pd in police_data_tables.keys():

    city_name = city_pd.removesuffix(' Police Department')  # cleaning this from end of string
    city_name = city_name.replace(' ', '_')                 # replace spaces with underscore
    
    # make some new data for the purposes of using in our GeoDataFrames
    total_traffic_stops = police_data_tables[city_pd].shape[0]    # get count of number of rows in dataframe (ie: number of incidents)
    stops_white = police_data_tables[city_pd].loc[police_data_tables[city_pd]['race'] == 'WHITE']
    stops_black = police_data_tables[city_pd].loc[police_data_tables[city_pd]['race'] == 'BLACK OR AFRICAN AMERICAN']
    
    # add police data to GeoDataFrame for each city
    myVars[city_name]['total traffic stops'] = total_traffic_stops
    myVars[city_name]['stops white'] = stops_white.shape[0]
    myVars[city_name]['stops black'] = stops_black.shape[0]
    myVars[city_name]['Pct White Stops'] = stops_white.shape[0] / total_traffic_stops
    myVars[city_name]['Pct Black Stops'] = stops_black.shape[0] / total_traffic_stops
    
    # all but Charlottesville tracks Terry Stops so I have to account for this with some error handling
    try:
        myVars[city_name]['Pct White Terry Stop'] = stops_white['reason_for_stop'].value_counts()['TERRY STOP'] / stops_white.shape[0]
        myVars[city_name]['Pct Black Terry Stop'] = stops_black['reason_for_stop'].value_counts()['TERRY STOP'] / stops_black.shape[0]
    except:
        myVars[city_name]['Pct White Terry Stop'] = 0.0
        myVars[city_name]['Pct Black Terry Stop'] = 0.0
        
    
print(Richmond['stops white'])
print(Richmond['Pct White Terry Stop'])



In [ ]:
# concatenate all the dataframes into one dataframe called 'all_VA'data'. This is for the purpose of plotting it all together
all_VA_data = pd.concat(cities)

fig, ax = plt.subplots(figsize=(15, 7))
all_VA_data.plot(ax=ax, column='Pct Black Stops', edgecolor='black', legend=True, cmap='OrRd')
fig.suptitle('% Black Stops', fontsize=25)
contextily.add_basemap(ax=ax, crs=all_VA_data.crs)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 7))
all_VA_data.plot(ax=ax, column='Pct White Stops', edgecolor='black', legend=True, cmap='OrRd')
fig.suptitle('% White Stops', fontsize=25)
contextily.add_basemap(ax=ax, crs=all_VA_data.crs)

In [ ]:

fig, ax = plt.subplots(figsize=(15, 7))
all_VA_data.plot(ax=ax, column='Pct White Terry Stop', edgecolor='black', legend=True, cmap='OrRd')
fig.suptitle('% White Terry Stops', fontsize=25)
contextily.add_basemap(ax=ax, crs=all_VA_data.crs)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 7))
all_VA_data.plot(ax=ax, column='Pct Black Terry Stop', edgecolor='black', legend=True, cmap='OrRd')
fig.suptitle('% Black Terry Stops', fontsize=25)
contextily.add_basemap(ax=ax, crs=all_VA_data.crs)

# Folium - Leaflet.js for python!

If anyone here ever uses Javascript, you might be familiar with the mapping library, [Leaflet](https://leafletjs.com/).

[Folium](https://python-visualization.github.io/folium/) is another python mapping library built on top of Leaflet. Now, we can make really nice web maps with python instead of Javascript! The strength of folium is in its mapping capabilities. This is not a data manipulation library in the way that GeopPandas is.

In [ ]:
m = folium.Map(location=[37.5407, -77.436], zoom_start=10)
m

### Add Data to the Map

We can add all the same kinds of data we previously used in GeoPandas

In [ ]:
# notice that I am streaming this data from a URL provided by the city of Richmond's data portal: https://richmond-geo-hub-cor.hub.arcgis.com/datasets/cor::public-libraries/about
richmond_libraries = f"https://services1.arcgis.com/k3vhq11XkBNeeOfM/arcgis/rest/services/PublicLibrary/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson"

m = folium.Map(
    location=[37.5407, -77.436],
    zoom_start=11,
)

folium.GeoJson(richmond_libraries, name="geojson").add_to(m)

m

### Choropleth maps

Folium has some really nice looking mapping capabilities. These go far beyond the plots of geopandas and matplotlib

In [ ]:
# I got this example straight from the Folium documentation: https://python-visualization.github.io/folium/quickstart.html#Choropleth-maps

url = (
    "https://raw.githubusercontent.com/python-visualization/folium/master/examples/data"
)
state_geo = f"{url}/us-states.json"
state_unemployment = f"{url}/US_Unemployment_Oct2012.csv"
state_data = pd.read_csv(state_unemployment)

m = folium.Map(location=[48, -102], zoom_start=3)

folium.Choropleth(
    geo_data=state_geo,
    name="choropleth",
    data=state_data,
    columns=["State", "Unemployment"],
    key_on="feature.id",
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Unemployment Rate (%)",
).add_to(m)

folium.LayerControl().add_to(m)

m

### Using GeoPandas with Folium

In [ ]:
#geopandas docs about plotting with folium: https://geopandas.org/en/stable/gallery/plotting_with_folium.html

volcanoes = pd.read_csv(my_path + '/volcano_data_2010.csv')

# keep only relevant columns
volcanoes = volcanoes.loc[:, ("Year", "Name", "Country", "Latitude", "Longitude", "Type")]
volcanoes

In [ ]:
# create point geometries for the volcanoes
geometry = gpd.points_from_xy(volcanoes['Longitude'], volcanoes['Latitude'])    # this makes point objects in an array

volcanoes['geometry'] = geometry     # add points as new column to geodataframe
volcanoes

In [ ]:
#make map with OpenStreetMap basemap

# OpenStreetMap
mymap = folium.Map(location = [0, 0], tiles='OpenStreetMap' , zoom_start = 1)
mymap

### Make Markers for Each Volcano in Folium Map

This is tricky stuff but basically we are iterating over each point in the GeoDataFrame and creating a marker for it.

In [ ]:
# Create Markers for Volcanoes

# Create a geometry list from the GeoDataFrame
geo_df_list = [[point.xy[1][0], point.xy[0][0]] for point in volcanoes.geometry ]

# Iterate through list and add a marker for each volcano, color-coded by its type.
i = 0
for coordinates in geo_df_list:
    #assign a color marker for the type of volcano, Strato being the most common
    if volcanoes.Type[i] == "Stratovolcano":
        type_color = "green"
    elif volcanoes.Type[i] == "Complex volcano":
        type_color = "blue"
    elif volcanoes.Type[i] == "Shield volcano":
        type_color = "orange"
    elif volcanoes.Type[i] == "Lava dome":
        type_color = "pink"
    else:
        type_color = "purple"


    # Place the markers with the popup labels and data
    mymap.add_child(folium.Marker(location = coordinates,
                            popup =
                                "Year: " + str(volcanoes.Year[i]) + '<br>' + '<br>' +
                                "Name: " + str(volcanoes.Name[i]) + '<br>' + '<br>' +
                                "Country: " + str(volcanoes.Country[i]) + '<br>' '<br>' +
                                "Type: " + str(volcanoes.Type[i]) + '<br>' '<br>' +
                                "Coordinates: " + str(geo_df_list[i]),
                            icon = folium.Icon(color = "%s" % type_color)))
    i+=1
    
mymap